<a href="https://colab.research.google.com/github/Ermiyas-03/01_my_repo1/blob/main/p1ch3/2_named_tensors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
_ = torch.tensor([0.2126, 0.7152, 0.0722], names=['c'])

<ipython-input-1-32e0d8fb07c9>:2: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at ../c10/core/TensorImpl.h:1928.)
  _ = torch.tensor([0.2126, 0.7152, 0.0722], names=['c'])


`torch.tensor(...)`

 This part creates a PyTorch tensor. A tensor is a multi-dimensional array, similar to a list or a NumPy array, but designed for efficient computation, especially on GPUs. It's the fundamental data structure used in PyTorch for deep learning.

`[0.2126, 0.7152, 0.0722]`

These are the values that are stored within the tensor. In this case, it's a 1-dimensional tensor (a vector) containing three floating-point numbers. These numbers likely represent weights or coefficients for some kind of calculation.

`names=['c']`

This is where we assign a name to the dimension of the tensor. In this case, the single dimension of the tensor is given the name 'c'. Using named dimensions can make your code more readable and help prevent errors when performing operations on tensors.

`_ =`

The underscore (_) is used in Python to indicate that we are not interested in storing the result of the operation in a specific variable. We are creating the tensor, but not assigning it to a named variable for later use. This might be done because the focus is more on creating the tensor with a named dimension for demonstration or internal purposes, rather than using it immediately in further calculations.

In [2]:
# Creating an image tensor
img_t = torch.randn(3, 5, 5) # shape [channels, rows, columns]

# Creating a weights tensor
weights = torch.tensor([0.2126, 0.7152, 0.0722])

`img_t` now holds a 3x5x5 tensor representing an image with 3 color channels and a 5x5 pixel grid, filled with random values.

In [ ]:
batch_t = torch.randn(2, 3, 5, 5) # shape [batch, channels, rows, columns]

In [ ]:
img_gray_naive = img_t.mean(-3)
batch_gray_naive = batch_t.mean(-3)
img_gray_naive.shape, batch_gray_naive.shape

(torch.Size([5, 5]), torch.Size([2, 5, 5]))

In [ ]:
unsqueezed_weights = weights.unsqueeze(-1).unsqueeze_(-1)
img_weights = (img_t * unsqueezed_weights)
batch_weights = (batch_t * unsqueezed_weights)
img_gray_weighted = img_weights.sum(-3)
batch_gray_weighted = batch_weights.sum(-3)
batch_weights.shape, batch_t.shape, unsqueezed_weights.shape

(torch.Size([2, 3, 5, 5]), torch.Size([2, 3, 5, 5]), torch.Size([3, 1, 1]))

In [ ]:
img_gray_weighted_fancy = torch.einsum('...chw,c->...hw', img_t, weights)
batch_gray_weighted_fancy = torch.einsum('...chw,c->...hw', batch_t, weights)
batch_gray_weighted_fancy.shape

torch.Size([2, 5, 5])

In [ ]:
weights_named = torch.tensor([0.2126, 0.7152, 0.0722], names=['channels'])
weights_named

tensor([0.2126, 0.7152, 0.0722], names=('channels',))

In [ ]:
img_named =  img_t.refine_names(..., 'channels', 'rows', 'columns')
batch_named = batch_t.refine_names(..., 'channels', 'rows', 'columns')
print("img named:", img_named.shape, img_named.names)
print("batch named:", batch_named.shape, batch_named.names)

img named: torch.Size([3, 5, 5]) ('channels', 'rows', 'columns')
batch named: torch.Size([2, 3, 5, 5]) (None, 'channels', 'rows', 'columns')


In [ ]:
weights_aligned = weights_named.align_as(img_named)
weights_aligned.shape, weights_aligned.names

(torch.Size([3, 1, 1]), ('channels', 'rows', 'columns'))

In [ ]:
gray_named = (img_named * weights_aligned).sum('channels')
gray_named.shape, gray_named.names

(torch.Size([5, 5]), ('rows', 'columns'))

In [ ]:
try:
    gray_named = (img_named[..., :3] * weights_named).sum('channels')
except Exception as e:
    print(e)

Error when attempting to broadcast dims ['channels', 'rows', 'columns'] and dims ['channels']: dim 'columns' and dim 'channels' are at the same position from the right but do not match.


In [ ]:
gray_plain = gray_named.rename(None)
gray_plain.shape, gray_plain.names

(torch.Size([5, 5]), (None, None))